In [16]:
import pandas as pd
import glob
import datetime

### Rube Goldberg to find the latest file

In [17]:
file_list = glob.glob("../data/raw/uw_monthly_reports/*.xlsx")

In [18]:
date_list = []
for f in file_list:
    raw_name = f.replace("../data/raw/uw_monthly_reports/","").replace(".xlsx","")
    clean_name = raw_name.replace("_","-")
    # ignore revision notes
    trim_name = clean_name.split("-rev")[0]
    # get file date
    file_date = trim_name.split("data")[1]  
    try:
        parsed_date = datetime.datetime.strptime(file_date, '%m%d%y')
    except ValueError:
        print(f"skip ```{raw_name}``` invalid date")
    
    date_list.append(parsed_date)

In [19]:
latest = max(date_list)
latest

datetime.datetime(2022, 5, 10, 0, 0)

In [20]:
latest_xl_file = f"../data/raw/uw_monthly_reports/uw_supplier_data{latest.strftime('%m%d%y')}.xlsx"
latest_xl_file

'../data/raw/uw_monthly_reports/uw_supplier_data051022.xlsx'

### Parse

In [21]:
xls = pd.ExcelFile(latest_xl_file)

In [22]:
xls.sheet_names

['Jun14-Mar22 Conservation Data',
 '2020 Baseline Values',
 'Jul21-Mar22 relative to 2020']

In [23]:
sheet_to_df_map = pd.read_excel(latest_xl_file, sheet_name=None)

First sheet contains residential water usage by supplier

In [24]:
usage_df = sheet_to_df_map[xls.sheet_names[0]]

Second sheet contains 2020 baselines

In [25]:
baselines_df = sheet_to_df_map[xls.sheet_names[1]]

Save this sheet's name for baseline filename in case it changes for some reason

In [26]:
baselines_name = xls.sheet_names[1].replace(" ","-").lower()

Third sheet conntains monthly usage and pct changes relative to 2020

In [27]:
change_df = sheet_to_df_map[xls.sheet_names[2]]

### Export

In [28]:
usage_df.to_csv("../data/raw/uw-usage.csv", index=False)

In [29]:
baselines_df.to_csv(f"../data/raw/uw-{baselines_name}.csv", index=False)

In [30]:
change_df.to_csv("../data/raw/uw-change-from-baseline.csv", index=False)